In [ ]:
import sys
from pathlib import Path

# Add root folder to Python path (to import modules)
notebook_dir = Path().absolute()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

In [ ]:
import warnings
import pandas as pd
import src.models.neural_net.metric as metric
from src.portfolios.portfolio_new import Portfolio
import torch
import torchmetrics
from os import listdir
from os.path import isfile, join
import numpy as np

Device: cpu
Epochs: 100
Random seed set at: 21


# Gu et al. 2020

## All networks from Grid search - Gu et al. 2020

In [3]:
gupath = '/home/ge65cuw/thesis/saved/results_old/NN3Ensemble/predicted_returns/'
gu_files_list = [f for f in listdir(gupath) if isfile(join(gupath, f))]

df_columns = columns=[
        'trial_id',
        'oosSpearman',
        'MDA', 
        'Turnover_long_VW', 
        'Turnover_short_VW', 
        'Turnover_long-short_VW', 
        'Max1MLoss_VW',
        'MaxDD_VW', 
        'Avg_ret_VW', 
        'Std_VW', 
        'Sharpe_ratio_VW', 
        'FF5_Mom_STRev_alpha_VW',
        'alpha_t-stat_VW',
        'regression_R2_VW', 
        'regression_Information_ratio_VW',
        'Turnover_long_EW', 
        'Turnover_short_EW', 
        'Turnover_long-short_EW', 
        'Max1MLoss_EW',
        'MaxDD_EW', 
        'Avg_ret_EW', 
        'Std_EW', 
        'Sharpe_ratio_EW', 
        'FF5_Mom_STRev_alpha_EW',
        'alpha_t-stat_EW', 
        'regression_R2_EW', 
        'regression_Information_ratio_EW']

import warnings
results_df = pd.DataFrame(columns=df_columns)
i=0
for file in gu_files_list:
        i+=1
        trial_id = file.split('_')[0] 
        results_dict = {}
        df = pd.read_csv(gupath + file, index_col=0)
        for column in df_columns:
                results_dict[column] = np.nan
        
        results_dict['trial_id'] = trial_id
        with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                results_dict['oosSpearman'] = metric.calc_spearman(torch.tensor(df['predicted_ret']), torch.tensor(df['ret'])).item()

        results_dict['MDA'] = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
        try:
                portfolioVW = Portfolio(df, weighting='VW', verbose = 0)
                results_dict['Turnover_long_VW'] = portfolioVW.turnover['long']
                results_dict['Turnover_short_VW'] = portfolioVW.turnover['short']
                results_dict['Turnover_long-short_VW'] = portfolioVW.turnover['long-short']
                results_dict['Max1MLoss_VW'] = portfolioVW.max1MLoss_long_short
                results_dict['MaxDD_VW'] = portfolioVW.maxDD_long_short
                results_dict['Avg_ret_VW'] = portfolioVW.average_long_short_ret
                results_dict['Std_VW'] = portfolioVW.std_dev_long_short
                results_dict['Sharpe_ratio_VW'] = portfolioVW.long_short_sharpe_ratio
                results_dict['FF5_Mom_STRev_alpha_VW'] = portfolioVW.alpha
                results_dict['alpha_t-stat_VW'] = portfolioVW.t_value_alpha
                results_dict['regression_R2_VW'] = portfolioVW.regression_rsquared
                results_dict['regression_Information_ratio_VW'] = portfolioVW.information_ratio_regression
        except Exception as e:
                print('Error in Value Weighted Portfolio')
                print(e)
        #print('EW portfolio')
        try:
                portfolioEW = Portfolio(df, weighting='EW', verbose = 0)
                results_dict['Turnover_long_EW'] = portfolioEW.turnover['long']
                results_dict['Turnover_short_EW'] = portfolioEW.turnover['short']
                results_dict['Turnover_long-short_EW'] = portfolioEW.turnover['long-short']
                results_dict['Max1MLoss_EW'] = portfolioEW.max1MLoss_long_short
                results_dict['MaxDD_EW'] = portfolioEW.maxDD_long_short
                results_dict['Avg_ret_EW'] = portfolioEW.average_long_short_ret
                results_dict['Std_EW'] = portfolioEW.std_dev_long_short
                results_dict['Sharpe_ratio_EW'] = portfolioEW.long_short_sharpe_ratio
                results_dict['FF5_Mom_STRev_alpha_EW'] = portfolioEW.alpha
                results_dict['alpha_t-stat_EW'] = portfolioEW.t_value_alpha
                results_dict['regression_R2_EW'] = portfolioEW.regression_rsquared
                results_dict['regression_Information_ratio_EW'] = portfolioEW.information_ratio_regression
        except Exception as e:
                print('Error with Equal Weighted Portfolio')
                print(e)
        print(f'Trial no.{i} [of {len(gu_files_list)}] completed')     
        results = pd.DataFrame(results_dict, index=[0])
        results_df = pd.concat([results_df, results], ignore_index=True)

results_df.to_csv('/home/ge65cuw/thesis/saved/final_results/gu_net_grid_search_results.csv')
results_df


KeyboardInterrupt: 

## Ensemble - Gu et al. 2020

In [3]:
results_df = pd.DataFrame()
results_dict = {}
df = pd.read_csv('/home/ge65cuw/thesis/saved/final_results/ensembleGu.csv', index_col=0)

with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        results_dict['oosSpearman'] = metric.calc_spearman(torch.tensor(df['predicted_ret']), torch.tensor(df['ret'])).item()

results_dict['MDA'] = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
try:
        portfolioVW = Portfolio(df, weighting='VW', verbose = 1)
        results_dict['Turnover_long_VW'] = portfolioVW.turnover['long']
        results_dict['Turnover_short_VW'] = portfolioVW.turnover['short']
        results_dict['Turnover_long-short_VW'] = portfolioVW.turnover['long-short']
        results_dict['Max1MLoss_VW'] = portfolioVW.max1MLoss_long_short
        results_dict['MaxDD_VW'] = portfolioVW.maxDD_long_short
        results_dict['Avg_ret_VW'] = portfolioVW.average_long_short_ret
        results_dict['Std_VW'] = portfolioVW.std_dev_long_short
        results_dict['Sharpe_ratio_VW'] = portfolioVW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_VW'] = portfolioVW.alpha
        results_dict['alpha_t-stat_VW'] = portfolioVW.t_value_alpha
        results_dict['regression_R2_VW'] = portfolioVW.regression_rsquared
        results_dict['regression_Information_ratio_VW'] = portfolioVW.information_ratio_regression
except Exception as e:
        print('Error in Value Weighted Portfolio')
        print(e)
#print('EW portfolio')
try:
        portfolioEW = Portfolio(df, weighting='EW', verbose = 0)
        results_dict['Turnover_long_EW'] = portfolioEW.turnover['long']
        results_dict['Turnover_short_EW'] = portfolioEW.turnover['short']
        results_dict['Turnover_long-short_EW'] = portfolioEW.turnover['long-short']
        results_dict['Max1MLoss_EW'] = portfolioEW.max1MLoss_long_short
        results_dict['MaxDD_EW'] = portfolioEW.maxDD_long_short
        results_dict['Avg_ret_EW'] = portfolioEW.average_long_short_ret
        results_dict['Std_EW'] = portfolioEW.std_dev_long_short
        results_dict['Sharpe_ratio_EW'] = portfolioEW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_EW'] = portfolioEW.alpha
        results_dict['alpha_t-stat_EW'] = portfolioEW.t_value_alpha
        results_dict['regression_R2_EW'] = portfolioEW.regression_rsquared
        results_dict['regression_Information_ratio_EW'] = portfolioEW.information_ratio_regression
except Exception as e:
        print('Error with Equal Weighted Portfolio')
        print(e)

results = pd.DataFrame(results_dict, index=[0])
pd.set_option('display.max_columns', None)
results

Loading the dataset...
Features stats:
Mean: -0.01	Std: 0.29	Median: -0.02
IR & alpha calculation...
Params
const     1.585319
Mkt-RF   -0.042643
SMB      -0.456151
HML       0.284871
RMW       0.433332
CMA       0.338306
Mom       0.134050
ST_Rev   -0.125761
dtype: float64
tValues
const     6.466020
Mkt-RF   -0.664069
SMB      -5.523790
HML       2.765142
RMW       4.010977
CMA       2.253171
Mom       2.532449
ST_Rev   -1.865450
dtype: float64
Linear Regression summary:
                            OLS Regression Results                            
Dep. Variable:                    l-s   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     28.51
Date:                Fri, 18 Nov 2022   Prob (F-statistic):           1.01e-29
Time:                        09:48:44   Log-Likelihood:                -816.21
No. Observations:                 296   AIC:     

,oosSpearman,MDA,Turnover_long_VW,Turnover_short_VW,Turnover_long-short_VW,Max1MLoss_VW,MaxDD_VW,Avg_ret_VW,Std_VW,Sharpe_ratio_VW,FF5_Mom_STRev_alpha_VW,alpha_t-stat_VW,regression_R2_VW,regression_Information_ratio_VW,Turnover_long_EW,Turnover_short_EW,Turnover_long-short_EW,Max1MLoss_EW,MaxDD_EW,Avg_ret_EW,Std_EW,Sharpe_ratio_EW,FF5_Mom_STRev_alpha_EW,alpha_t-stat_EW,regression_R2_EW,regression_Information_ratio_EW
0,0.076688,0.524872,151.315893,124.287184,275.603077,13.454363,0.400844,1.854805,4.96168,0.373826,1.585319,6.46602,0.409289,0.360337,22.317198,26.759792,49.07699,9.221077,0.111946,0.87637,2.007644,0.436517,0.710001,8.739815,0.603892,0.487051


# Linear benchmark 

In [6]:
linearPredictedRet = '/home/ge65cuw/thesis/saved/final_results/linear/predicted_ret.csv'

In [20]:
df = pd.read_csv(linearPredictedRet, index_col=0)
df = df.loc[df.yyyymm >= 200001]
df2 = pd.read_csv('/home/ge65cuw/thesis/saved/final_results/ensembleGu.csv', index_col=0)
print(df.shape, df2.shape)

(1670935, 4) (473018, 4)


In [26]:
import torch
t1 = torch.tensor(df['predicted_ret'].to_numpy())
t2 = torch.tensor(df['ret'].to_numpy())
spearman = metric.calc_spearman(t1,t2).item()
spearman

/home/ge65cuw/anaconda3/envs/thesis/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


0.05678696720163356

In [28]:
MDA = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
MDA

0.5190160000239387

In [8]:
results_df = pd.DataFrame()
results_dict = {}
df = pd.read_csv(linearPredictedRet, index_col=0)

with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        results_dict['oosSpearman'] = metric.calc_spearman(torch.tensor(df['predicted_ret']), torch.tensor(df['ret'])).item()

results_dict['MDA'] = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
try:
        portfolioVW = Portfolio(df, weighting='VW', verbose = 0)
        results_dict['Turnover_long_VW'] = portfolioVW.turnover['long']
        results_dict['Turnover_short_VW'] = portfolioVW.turnover['short']
        results_dict['Turnover_long-short_VW'] = portfolioVW.turnover['long-short']
        results_dict['Max1MLoss_VW'] = portfolioVW.max1MLoss_long_short
        results_dict['MaxDD_VW'] = portfolioVW.maxDD_long_short
        results_dict['Avg_ret_VW'] = portfolioVW.average_long_short_ret
        results_dict['Std_VW'] = portfolioVW.std_dev_long_short
        results_dict['Sharpe_ratio_VW'] = portfolioVW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_VW'] = portfolioVW.alpha
        results_dict['alpha_t-stat_VW'] = portfolioVW.t_value_alpha
        results_dict['regression_R2_VW'] = portfolioVW.regression_rsquared
        results_dict['regression_Information_ratio_VW'] = portfolioVW.information_ratio_regression
except Exception as e:
        print('Error in Value Weighted Portfolio')
        print(e)
#print('EW portfolio')
try:
        portfolioEW = Portfolio(df, weighting='EW', verbose = 0)
        results_dict['Turnover_long_EW'] = portfolioEW.turnover['long']
        results_dict['Turnover_short_EW'] = portfolioEW.turnover['short']
        results_dict['Turnover_long-short_EW'] = portfolioEW.turnover['long-short']
        results_dict['Max1MLoss_EW'] = portfolioEW.max1MLoss_long_short
        results_dict['MaxDD_EW'] = portfolioEW.maxDD_long_short
        results_dict['Avg_ret_EW'] = portfolioEW.average_long_short_ret
        results_dict['Std_EW'] = portfolioEW.std_dev_long_short
        results_dict['Sharpe_ratio_EW'] = portfolioEW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_EW'] = portfolioEW.alpha
        results_dict['alpha_t-stat_EW'] = portfolioEW.t_value_alpha
        results_dict['regression_R2_EW'] = portfolioEW.regression_rsquared
        results_dict['regression_Information_ratio_EW'] = portfolioEW.information_ratio_regression
except Exception as e:
        print('Error with Equal Weighted Portfolio')
        print(e)

results = pd.DataFrame(results_dict, index=[0])
pd.set_option('display.max_columns', None)
results

KeyboardInterrupt: 

# All results from NN3 Grid Search

In [ ]:
nn3path = '/home/ge65cuw/thesis/saved/results_old/NN3Ensemble/predicted_returns/'
nn3_files_list = [f for f in listdir(nn3path) if isfile(join(nn3path, f))]

In [ ]:
df_columns = columns=[
        'trial_id',
        'oosSpearman',
        'MDA', 
        'Turnover_long_VW', 
        'Turnover_short_VW', 
        'Turnover_long-short_VW', 
        'Max1MLoss_VW',
        'MaxDD_VW', 
        'Avg_ret_VW', 
        'Std_VW', 
        'Sharpe_ratio_VW', 
        'FF5_Mom_STRev_alpha_VW',
        'alpha_t-stat_VW',
        'regression_R2_VW', 
        'regression_Information_ratio_VW',
        'Turnover_long_EW', 
        'Turnover_short_EW', 
        'Turnover_long-short_EW', 
        'Max1MLoss_EW',
        'MaxDD_EW', 
        'Avg_ret_EW', 
        'Std_EW', 
        'Sharpe_ratio_EW', 
        'FF5_Mom_STRev_alpha_EW',
        'alpha_t-stat_EW', 
        'regression_R2_EW', 
        'regression_Information_ratio_EW']

import warnings
results_df = pd.DataFrame(columns=df_columns)
i=0
for file in gu_files_list:
        i+=1
        trial_id = file.split('_')[0] 
        results_dict = {}
        df = pd.read_csv(gupath + file, index_col=0)
        for column in df_columns:
                results_dict[column] = np.nan
        
        results_dict['trial_id'] = trial_id
        with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                results_dict['oosSpearman'] = metric.calc_spearman(torch.tensor(df['predicted_ret']), torch.tensor(df['ret'])).item()

        results_dict['MDA'] = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
        try:
                portfolioVW = Portfolio(df, weighting='VW', verbose = 0)
                results_dict['Turnover_long_VW'] = portfolioVW.turnover['long']
                results_dict['Turnover_short_VW'] = portfolioVW.turnover['short']
                results_dict['Turnover_long-short_VW'] = portfolioVW.turnover['long-short']
                results_dict['Max1MLoss_VW'] = portfolioVW.max1MLoss_long_short
                results_dict['MaxDD_VW'] = portfolioVW.maxDD_long_short
                results_dict['Avg_ret_VW'] = portfolioVW.average_long_short_ret
                results_dict['Std_VW'] = portfolioVW.std_dev_long_short
                results_dict['Sharpe_ratio_VW'] = portfolioVW.long_short_sharpe_ratio
                results_dict['FF5_Mom_STRev_alpha_VW'] = portfolioVW.alpha
                results_dict['alpha_t-stat_VW'] = portfolioVW.t_value_alpha
                results_dict['regression_R2_VW'] = portfolioVW.regression_rsquared
                results_dict['regression_Information_ratio_VW'] = portfolioVW.information_ratio_regression
        except Exception as e:
                print('Error in Value Weighted Portfolio')
                print(e)
        #print('EW portfolio')
        try:
                portfolioEW = Portfolio(df, weighting='EW', verbose = 0)
                results_dict['Turnover_long_EW'] = portfolioEW.turnover['long']
                results_dict['Turnover_short_EW'] = portfolioEW.turnover['short']
                results_dict['Turnover_long-short_EW'] = portfolioEW.turnover['long-short']
                results_dict['Max1MLoss_EW'] = portfolioEW.max1MLoss_long_short
                results_dict['MaxDD_EW'] = portfolioEW.maxDD_long_short
                results_dict['Avg_ret_EW'] = portfolioEW.average_long_short_ret
                results_dict['Std_EW'] = portfolioEW.std_dev_long_short
                results_dict['Sharpe_ratio_EW'] = portfolioEW.long_short_sharpe_ratio
                results_dict['FF5_Mom_STRev_alpha_EW'] = portfolioEW.alpha
                results_dict['alpha_t-stat_EW'] = portfolioEW.t_value_alpha
                results_dict['regression_R2_EW'] = portfolioEW.regression_rsquared
                results_dict['regression_Information_ratio_EW'] = portfolioEW.information_ratio_regression
        except Exception as e:
                print('Error with Equal Weighted Portfolio')
                print(e)
        print(f'Trial no.{i} [of {len(gu_files_list)}] completed')     
        results = pd.DataFrame(results_dict, index=[0])
        results_df = pd.concat([results_df, results], ignore_index=True)

results_df.to_csv('/home/ge65cuw/thesis/saved/final_results/best_net_grid_search_results.csv')
results_df

Remove MaxDD printing and implement logic to not save networks which fail at creating portfolios
Trial no.1 [of 10] completed
Trial no.2 [of 10] completed
Trial no.3 [of 10] completed
Trial no.4 [of 10] completed
Trial no.5 [of 10] completed
Trial no.6 [of 10] completed
Trial no.7 [of 10] completed
Trial no.8 [of 10] completed
Trial no.9 [of 10] completed
Trial no.10 [of 10] completed


## Ensemble results

In [4]:
results_df = pd.DataFrame()
results_dict = {}
df = pd.read_csv('/home/ge65cuw/thesis/saved/final_results/ensembleNN3.csv', index_col=0)

with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        results_dict['oosSpearman'] = metric.calc_spearman(torch.tensor(df['predicted_ret']), torch.tensor(df['ret'])).item()

results_dict['MDA'] = metric.mean_directional_accuracy(df['predicted_ret'], df['ret'])
try:
        portfolioVW = Portfolio(df, weighting='VW', verbose = 1)
        results_dict['Turnover_long_VW'] = portfolioVW.turnover['long']
        results_dict['Turnover_short_VW'] = portfolioVW.turnover['short']
        results_dict['Turnover_long-short_VW'] = portfolioVW.turnover['long-short']
        results_dict['Max1MLoss_VW'] = portfolioVW.max1MLoss_long_short
        results_dict['MaxDD_VW'] = portfolioVW.maxDD_long_short
        results_dict['Avg_ret_VW'] = portfolioVW.average_long_short_ret
        results_dict['Std_VW'] = portfolioVW.std_dev_long_short
        results_dict['Sharpe_ratio_VW'] = portfolioVW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_VW'] = portfolioVW.alpha
        results_dict['alpha_t-stat_VW'] = portfolioVW.t_value_alpha
        results_dict['regression_R2_VW'] = portfolioVW.regression_rsquared
        results_dict['regression_Information_ratio_VW'] = portfolioVW.information_ratio_regression
except Exception as e:
        print('Error in Value Weighted Portfolio')
        print(e)
#print('EW portfolio')
try:
        portfolioEW = Portfolio(df, weighting='EW', verbose = 0)
        results_dict['Turnover_long_EW'] = portfolioEW.turnover['long']
        results_dict['Turnover_short_EW'] = portfolioEW.turnover['short']
        results_dict['Turnover_long-short_EW'] = portfolioEW.turnover['long-short']
        results_dict['Max1MLoss_EW'] = portfolioEW.max1MLoss_long_short
        results_dict['MaxDD_EW'] = portfolioEW.maxDD_long_short
        results_dict['Avg_ret_EW'] = portfolioEW.average_long_short_ret
        results_dict['Std_EW'] = portfolioEW.std_dev_long_short
        results_dict['Sharpe_ratio_EW'] = portfolioEW.long_short_sharpe_ratio
        results_dict['FF5_Mom_STRev_alpha_EW'] = portfolioEW.alpha
        results_dict['alpha_t-stat_EW'] = portfolioEW.t_value_alpha
        results_dict['regression_R2_EW'] = portfolioEW.regression_rsquared
        results_dict['regression_Information_ratio_EW'] = portfolioEW.information_ratio_regression
except Exception as e:
        print('Error with Equal Weighted Portfolio')
        print(e)

nn3_results = pd.DataFrame(results_dict, index=[0])
pd.set_option('display.max_columns', None)
nn3_results

Loading the dataset...
Features stats:
Mean: -0.01	Std: 0.29	Median: -0.02
IR & alpha calculation...
Params
const     1.390458
Mkt-RF   -0.045262
SMB      -0.157068
HML       0.346980
RMW       0.138406
CMA       0.260275
Mom       0.018406
ST_Rev   -0.006553
dtype: float64
tValues
const     6.178562
Mkt-RF   -0.767920
SMB      -2.072170
HML       3.669293
RMW       1.395709
CMA       1.888540
Mom       0.378823
ST_Rev   -0.105896
dtype: float64
Linear Regression summary:
                            OLS Regression Results                            
Dep. Variable:                    l-s   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     12.13
Date:                Fri, 18 Nov 2022   Prob (F-statistic):           1.41e-13
Time:                        09:56:12   Log-Likelihood:                -790.85
No. Observations:                 296   AIC:     

,oosSpearman,MDA,Turnover_long_VW,Turnover_short_VW,Turnover_long-short_VW,Max1MLoss_VW,MaxDD_VW,Avg_ret_VW,Std_VW,Sharpe_ratio_VW,FF5_Mom_STRev_alpha_VW,alpha_t-stat_VW,regression_R2_VW,regression_Information_ratio_VW,Turnover_long_EW,Turnover_short_EW,Turnover_long-short_EW,Max1MLoss_EW,MaxDD_EW,Avg_ret_EW,Std_EW,Sharpe_ratio_EW,FF5_Mom_STRev_alpha_EW,alpha_t-stat_EW,regression_R2_EW,regression_Information_ratio_EW
0,0.071609,0.523587,143.23767,131.071197,274.308867,9.047608,0.475057,1.514119,3.982943,0.380151,1.390458,6.178562,0.227664,0.344318,21.82325,25.251775,47.075025,9.27322,0.121359,0.908055,2.049344,0.443095,0.734495,9.04262,0.619956,0.503926
